# Практическая работа №1
## Предварительная обработка данных
Выполнил: Терентьев Никита Александрович КИ21-16/1б
Вариант: 21
## Цель работы
Знакомство с основными задачами предварительной обработки исходных данных, изучение основных методов предварительной обработки данных, формирование навыков выполнения предварительной обработки исходных данных с помощью языка программирования Python.
## Задачи
Выполнение практической работы предполагает решение следующих задач:
1. Визуальный анализ исходных данных
2. Поиск аномальных значений
3. Поиск и восстановление отсутствующих значений
4. Преобразование данных
## Ход работы
Был произведён импорт необходимых библиотек.

In [1]:
import pandas as pd
import numpy as np
import sklearn.impute

from scipy.stats import sigmaclip
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, StandardScaler

from draw_functions import build_bar_and_pie_chart, build_histogram_density_diagram

AttributeError: module 'numpy.linalg.linalg' has no attribute '__all__'

Произведено чтение файла и взятие нужных колонок из таблицы в соответствии с вариантом.

In [ ]:
dataframe = pd.read_excel(r"C:\Users\nikit\Desktop\Программирование\4 семестр\data_analysis_methods\1 lab\data\first_practice_dataframe.xlsx")
# Названия колонок датафрейма храним в константах
qualitative_variables = ["cat1_gender", "cat3_education"]
quantitative_variables = ["num1_22", "num2_22", "num3_22"]
dataframe = dataframe[[*qualitative_variables, *quantitative_variables]]
initial_dataframe = dataframe.copy()
dataframe

Выведем типы данных в нашем датафрейме.

In [ ]:
dataframe.dtypes

### 1. Визуальный анализ данных
Были построены графики для визуального представления каждого столбца(признака) в исходном наборе данных.
Все функции для построения графиков были вынесены в отдельный файл functions.py

In [ ]:
for column in qualitative_variables:
    build_bar_and_pie_chart(dataframe[column], column)

In [ ]:
for column in quantitative_variables:
    try:
        build_histogram_density_diagram(dataframe[column],column)
    except TypeError:
        print(f"График признака {column} не может быть построен, так как столбец содержит не только числовые значения.")

### 2. Проверка правдоподобности исходных данных
Проанализировав полученные графики, можно сделать вывод, что некоторые значения из таблицы являются некорректными (например, пропуски значений).

#### Приведение значений качественных признаков
Приведём значения качественных признаков к нужному формату (всем пустым значениям был присвоен тип None, признаки, отличающиеся типом написания, были приведены к одному виду). Также первая и вторая колонка были приведены к категориальному типу.

In [ ]:
for column in qualitative_variables:
    dataframe[column].replace({" ": None, "-": None, np.nan: None}, inplace=True)
    dataframe[column] = dataframe[column].apply(lambda x: x.capitalize() if isinstance(x, str) else x)
    dataframe[column] = dataframe[column].astype("category")

#### Приведение значений количественных признаков

In [ ]:
for column in quantitative_variables:
    # Если errors='coerce', то недопустимый синтаксический анализ будет установлен как NaN.
    dataframe[column] = pd.to_numeric(dataframe[column], errors='coerce')

In [ ]:
dataframe.dtypes

Выведем ещё раз все графики.

In [ ]:
for column in qualitative_variables:
    build_bar_and_pie_chart(dataframe[column], column)

In [ ]:
for column in quantitative_variables:
    try:
        build_histogram_density_diagram(dataframe[column],column)
    except TypeError:
        print(f"График признака {column} не может быть построен, так как столбец содержит не только числовые значения.")

Все данные были приведены к нужному формату. Все качественные признаки имеют тип category, все количественные - float64.

### 3. Поиск аномальных значений
Было произведено копирование датафрейма.

In [ ]:
dataframe_copy = dataframe.copy()
dataframe_sigma = dataframe.copy()
dataframe_quantile = dataframe.copy()

#### Метод сигм
Выведем изначальное количество значений

In [ ]:
for column in quantitative_variables:
    print(f"Количество не пустых значений в параметре {column}: {dataframe_sigma[column].count()}")

In [ ]:
def sigma_method(dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    Удаляет выбросы из числовых столбцов фрейма данных, используя метод сигма.
    :param dataframe: Датафрейм
    :return: Датафрейм после удаления выбросов.
    """
    numerical_columns = dataframe.select_dtypes(include=['float']).columns
    for column in numerical_columns:
        data = dataframe[column].dropna()
        clean_data, low, high = sigmaclip(data, low=3, high=3)
        dataframe = dataframe.loc[dataframe.loc[:, column].isin(clean_data) | dataframe.loc[:, column].isna()]
    return dataframe.loc[dataframe.loc[:, numerical_columns[0]].notna()]

dataframe_sigma = sigma_method(dataframe_sigma)

Выведем количество значений после удаления выбросов c помощью метода сигм.

In [ ]:
for column in quantitative_variables:
    print(f"Количество не пустых значений в параметре {column}: {dataframe_sigma[column].count()}")

In [ ]:
for column in quantitative_variables:
    try:
        build_histogram_density_diagram(dataframe_sigma[column],column)
    except TypeError:
        print(f"График признака {column} не может быть построен, так как столбец содержит не только числовые значения.")

#### Метод квантилей

In [ ]:
def quantile_method(dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    Метод квантилей
    :param dataframe: Датафрейм
    :return: измененный Датафрейм с удалёнными выбросами
    """
    num_colums = dataframe.select_dtypes(include=['float']).columns
    q25 = dataframe[num_colums].quantile(0.25)
    q75 = dataframe[num_colums].quantile(0.75)
    delta = q75 - q25
    low = q25 - 1.5 * delta
    high = q75 + 1.5 * delta
    filtered_dataframe = dataframe[
        ~((dataframe[num_colums] < low) | (dataframe[num_colums] > high)).any(axis=1)]
    filtered_dataframe = filtered_dataframe.reset_index(drop=True)
    return filtered_dataframe

dataframe_quantile = quantile_method(dataframe_quantile)

In [ ]:
for column in quantitative_variables:
    print(f"Количество не пустых значений в параметре {column}: {dataframe_quantile[column].count()}")

In [ ]:
for column in quantitative_variables:
    try:
        build_histogram_density_diagram(dataframe_quantile[column],column)
    except TypeError:
        print(f"График признака {column} не может быть построен, так как столбец содержит не только числовые значения.")

Используем результаты очистки данных, полученных с помощью метода сигм.

In [ ]:
dataframe = dataframe_sigma

### 4. Поиск и восстановление пропущенных значений
Узнаем количество пропущенных значений в нашем датафрейме.

In [ ]:
dataframe.info()

Всего в датафрейме 893 строки, в 3 столбце столько же значений, следовательно, в нём не надо восстанавливать пропущенные значения. Пропуски значений первого столбца будут заменены значениями, полученными с помощью метода “k-ближайших соседей”.

In [ ]:
first_column = qualitative_variables[0]

dataframe_filled = dataframe.copy()

nan_index = dataframe_filled[first_column][dataframe_filled[first_column].isna()].index[0]
imputer = sklearn.impute.KNNImputer(n_neighbors=5)
encoder = LabelEncoder()
dataframe_filled[first_column] = encoder.fit_transform(dataframe_filled[first_column])
dataframe_filled[first_column] = dataframe_filled[first_column].replace({dataframe_filled[first_column][nan_index]: np.nan})
dataframe_filled[first_column] = imputer.fit_transform(dataframe_filled[[first_column]])
dataframe_filled[first_column] = dataframe_filled[first_column].apply(lambda x: round(x))
dataframe_filled[first_column] = encoder.inverse_transform(dataframe_filled[first_column])
dataframe = dataframe_filled
print(f'Есть ли nan элементы: {dataframe[first_column].isna().any()}')

Пропуски значений второго признака были заменены самым популярным значением.

In [ ]:
second_column = qualitative_variables[1]
most_frequence = dataframe[second_column].value_counts().keys()[0]
dataframe[second_column] = dataframe[second_column].fillna(most_frequence)
print(f'Есть ли в столбце nan элементы: {dataframe[second_column].isna().any()}')

Пропуски значений четвёртого признака были заменены медианой.

In [ ]:
four_column = quantitative_variables[1]
median = dataframe[four_column].median()
dataframe[four_column] = dataframe[four_column].fillna(median)
print(f'Есть ли в столбце nan элементы: {dataframe[four_column].isna().any()}')

Пропуски значений пятого признака были заменены значениями, полученными методом "k-ближайших соседей".

In [ ]:
fifth_column = quantitative_variables[2]
knn_imputer = sklearn.impute.KNNImputer(n_neighbors=3)
# Заполняем пропущенные значения в выбранном столбце
dataframe[[fifth_column]] = knn_imputer.fit_transform(dataframe[[fifth_column]])
print(f'Есть ли в столбце nan элементы: {dataframe[fifth_column].isna().any()}')

Все значения были восстановлены.

### 5. Преобразование данных
#### Кодировка категориальных признаков
Было выполнена кодировка первого категориального признака с помощью one-hot encoding.

In [ ]:
onehot_encoder = OneHotEncoder(min_frequency=6, sparse_output=False, categories="auto")
dataframe_encoded = pd.DataFrame(onehot_encoder.fit_transform(dataframe[[qualitative_variables[0]]]), columns=onehot_encoder.get_feature_names_out())
dataframe = dataframe.join(dataframe_encoded)

Для второго категориального признака была произведена кодировка при помощи label encoding.

In [ ]:
dataframe[qualitative_variables[1] + "lb"] = LabelEncoder().fit_transform(dataframe[qualitative_variables[1]])
dataframe

#### Преобразование количественных признаков
В нормализованном наборе данных значения находятся между 0 и 1. Стандартизированный набор данных имеет нулевое среднее значение и единичную дисперсию (стандартное отклонение). Первый и второй количественный признак были приведены к нормализованному виду.

In [ ]:
min_max_scaler = MinMaxScaler()
dataframe_scaled = pd.DataFrame(min_max_scaler.fit_transform(dataframe[quantitative_variables]), columns=quantitative_variables)
dataframe = pd.merge(dataframe, dataframe_scaled, left_index=True, right_index=True, suffixes=('', '_norm'))

Третий количественный признак был приведён к стандартизированному виду.

In [ ]:
standart_scaler = StandardScaler()
dataframe_scaled = pd.DataFrame(standart_scaler.fit_transform(dataframe[quantitative_variables]), columns=quantitative_variables)
dataframe = pd.merge(dataframe, dataframe_scaled, left_index=True, right_index=True, suffixes=('', '_stand'))
dataframe

## Выводы
В ходе работы были изучены основные задачи предварительной обработки исходных данных и методы предварительной обработки данных. Был произведён визуальный анализ исходных данных, поиск аномальных значений, восстановление отсутствующих значений и преобразование данных. Ниже произведено сравнение графиков, построенных по исходным данным и по полученным в результате работы обработанным данным.

Выведем итоговые графики

In [ ]:
for column in qualitative_variables:
    build_bar_and_pie_chart(dataframe[column], column)

In [ ]:
for column in quantitative_variables:
    build_histogram_density_diagram(dataframe[column],column)